In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.io as pio
import plotly.graph_objects as go
pio.renderers.default = "iframe"

from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans, DBSCAN

In [2]:
pd.read_csv('taxi-zone-lookup.csv')

,LocationID,Borough,Zone
0,1,EWR,Newark Airport
1,2,Queens,Jamaica Bay
2,3,Bronx,Allerton/Pelham Gardens
3,4,Manhattan,Alphabet City
4,5,Staten Island,Arden Heights
...,...,...,...
260,261,Manhattan,World Trade Center
261,262,Manhattan,Yorkville East
262,263,Manhattan,Yorkville West
263,264,Unknown,Unknown


In [3]:
df_0414 = pd.read_csv('uber-raw-data-apr14.csv')

In [4]:
df_0514 = pd.read_csv('uber-raw-data-may14.csv')

In [5]:
df_0614 = pd.read_csv('uber-raw-data-jun14.csv')

In [6]:
df_0714 = pd.read_csv('uber-raw-data-jul14.csv')

In [7]:
df_0814 = pd.read_csv('uber-raw-data-aug14.csv')

In [8]:
df_0914 = pd.read_csv('uber-raw-data-sep14.csv')

In [9]:
df_010615 = pd.read_csv('uber-raw-data-janjune-15.csv')

In [10]:
df_2014 = pd.concat([df_0414,df_0514,df_0614,df_0714,df_0814,df_0914])

In [11]:
df_2014.head()

,Date/Time,Lat,Lon,Base
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512


In [12]:
df_2014.describe(include='all')

,Date/Time,Lat,Lon,Base
count,4534327,4.534327e+06,4.534327e+06,4534327
unique,260093,NaN,NaN,5
top,4/7/2014 20:21:00,NaN,NaN,B02617
freq,97,NaN,NaN,1458853
mean,NaN,4.073926e+01,-7.397302e+01,NaN
std,NaN,3.994991e-02,5.726670e-02,NaN
min,NaN,3.965690e+01,-7.492900e+01,NaN
25%,NaN,4.072110e+01,-7.399650e+01,NaN
50%,NaN,4.074220e+01,-7.398340e+01,NaN
75%,NaN,4.076100e+01,-7.396530e+01,NaN


In [13]:
df_2014['Date/Time'] = pd.to_datetime(df_2014['Date/Time'])
df_2014['hour'] = df_2014['Date/Time'].dt.hour
df_2014['month'] = df_2014['Date/Time'].dt.month
df_2014['day'] = df_2014['Date/Time'].dt.day
df_2014['dayofweek'] = df_2014['Date/Time'].dt.day_name()

In [14]:
df_2014.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4534327 entries, 0 to 1028135
Data columns (total 8 columns):
 #   Column     Dtype         
---  ------     -----         
 0   Date/Time  datetime64[ns]
 1   Lat        float64       
 2   Lon        float64       
 3   Base       object        
 4   hour       int32         
 5   month      int32         
 6   day        int32         
 7   dayofweek  object        
dtypes: datetime64[ns](1), float64(2), int32(3), object(2)
memory usage: 259.5+ MB


In [15]:
df_2014.nunique()

Date/Time    260093
Lat            7092
Lon           11453
Base              5
hour             24
month             6
day              31
dayofweek         7
dtype: int64

In [16]:
hourly_avg = df_2014.groupby(['hour','dayofweek'])['Date/Time'].count().reset_index()
avg_day_hour = hourly_avg.groupby(['dayofweek','hour'])['Date/Time'].mean().reset_index()

fig= px.line(data_frame=avg_day_hour, x='hour',y='Date/Time', color='dayofweek', markers=True,
             category_orders={'dayofweek':['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']},
             title='Average number of rides per hour by day of week')

fig.update_layout(
    xaxis = dict(tickvals = avg_day_hour['hour']),
    yaxis_title="Average number of rides",
    xaxis_title="Hours"
    
)
fig.show()

Two peaks can be highlighted : One between 5-6 AM and 9 AM and another one from 4 to 9 PM. Besides, Saturdays and Sundays are the only days with quite high average number of rides during the night compared to weekdays. That makes sense with weekend activities on Friday and Saturday evenings/nights.

In [17]:
#A sample is needed to be able to map correctly the data and avoid a crash due to lack of memory
sample = df_2014.sample(n=10000, weights='hour', random_state=0)
fig_base = px.scatter_mapbox(sample, lat='Lat',lon='Lon', color='Base',zoom=9,mapbox_style="carto-positron",width=800,height=800,title='Map of Uber pickups (sample) in 2014 by Base')
fig_base.show()


In [18]:
fig_hour = px.scatter_mapbox(sample, lat='Lat',lon='Lon', color='hour',zoom=9,mapbox_style="carto-positron",width=800,height=800,title='Map of Uber pickups (sample) in 2014 per hour')
fig_hour.show()

In [19]:
sample['Base'].value_counts()

Base
B02617    3168
B02598    3068
B02682    2678
B02764     600
B02512     486
Name: count, dtype: int64

In [20]:
sample[['Lat','Lon']]

,Lat,Lon
587300,40.8221,-73.9555
551923,40.7706,-73.9601
38471,40.7432,-74.0078
570574,40.7592,-73.9792
21778,40.6419,-73.7883
...,...,...
53018,40.7700,-73.9672
1011718,40.7767,-73.9831
313631,40.7708,-73.9670
448037,40.6877,-73.9781


In [21]:
sample

,Date/Time,Lat,Lon,Base,hour,month,day,dayofweek
587300,2014-07-31 19:59:00,40.8221,-73.9555,B02617,19,7,31,Thursday
551923,2014-08-26 18:43:00,40.7706,-73.9601,B02617,18,8,26,Tuesday
38471,2014-08-01 19:53:00,40.7432,-74.0078,B02598,19,8,1,Friday
570574,2014-07-30 16:37:00,40.7592,-73.9792,B02617,16,7,30,Wednesday
21778,2014-07-21 14:45:00,40.6419,-73.7883,B02512,14,7,21,Monday
...,...,...,...,...,...,...,...,...
53018,2014-04-04 17:24:00,40.7700,-73.9672,B02598,17,4,4,Friday
1011718,2014-09-28 16:35:00,40.7767,-73.9831,B02764,16,9,28,Sunday
313631,2014-07-06 12:36:00,40.7708,-73.9670,B02617,12,7,6,Sunday
448037,2014-06-28 22:56:00,40.6877,-73.9781,B02617,22,6,28,Saturday


In [22]:
# Import Standard Scaler
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
coord = ['Lat','Lon']
X = sc.fit_transform(sample[coord])

## **Clustering with KMeans**

In [23]:
wcss =  []
k = []
for i in range (1,15):
    kmeans = KMeans(n_clusters= i, random_state = 0, n_init = 'auto')
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
    k.append(i)
    print("WCSS for K={} --> {}".format(i, wcss[-1]))

C:\Users\j.lopes\AppData\Local\anaconda\Lib\site-packages\joblib\externals\loky\backend\context.py:110: UserWarning:

Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.

  File "C:\Users\j.lopes\AppData\Local\anaconda\Lib\site-packages\joblib\externals\loky\backend\context.py", line 217, in _count_physical_cores
    raise ValueError(


WCSS for K=1 --> 20000.0
WCSS for K=2 --> 14146.875810568019
WCSS for K=3 --> 11536.779990863502
WCSS for K=4 --> 8212.646734256568
WCSS for K=5 --> 7100.692869295192
WCSS for K=6 --> 6095.3730203944415
WCSS for K=7 --> 5418.989647276246
WCSS for K=8 --> 4041.571758793327
WCSS for K=9 --> 3522.17468032072
WCSS for K=10 --> 3173.483420510236
WCSS for K=11 --> 2804.5900861470286
WCSS for K=12 --> 2416.6766335392067
WCSS for K=13 --> 2202.636280582832
WCSS for K=14 --> 2087.4758898930795


In [24]:
# Create DataFrame
wcss_frame = pd.DataFrame(wcss)
k_frame = pd.Series(k)

# Create figure
fig= px.line(
    wcss_frame,
    x=k_frame,
    y=wcss_frame.iloc[:,-1]
)

# Create title and axis labels
fig.update_layout(
    yaxis_title="Inertia",
    xaxis_title="Number of clusters",
    title="Inertia per cluster"
)

# Render
#fig.show(renderer="notebook")
fig.show() # if using workspace

In [25]:
# Computer mean silhouette score
sil = []
k = []

## Careful, you need to start at i=2 as silhouette score cannot accept less than 2 labels
for i in range (2,15):
    kmeans = KMeans(n_clusters= i, random_state = 0, n_init = 'auto')
    kmeans.fit(X)
    sil.append(silhouette_score(X, kmeans.predict(X)))
    k.append(i)
    print("Silhouette score for K={} is {}".format(i, sil[-1]))

Silhouette score for K=2 is 0.7265085137348773
Silhouette score for K=3 is 0.564320609832981
Silhouette score for K=4 is 0.43029336812553054
Silhouette score for K=5 is 0.41757982600387533
Silhouette score for K=6 is 0.4495454186094687
Silhouette score for K=7 is 0.4544998125308262
Silhouette score for K=8 is 0.4786588802921833
Silhouette score for K=9 is 0.4812444173370239
Silhouette score for K=10 is 0.4870065168832287
Silhouette score for K=11 is 0.4469816824790004
Silhouette score for K=12 is 0.450837201819232
Silhouette score for K=13 is 0.4155468270544673
Silhouette score for K=14 is 0.41617969910888913


In [26]:
# Create a data frame
cluster_scores=pd.DataFrame(sil)
k_frame = pd.Series(k)

# Create figure
fig = px.bar(data_frame=cluster_scores,
             x=k,
             y=cluster_scores.iloc[:, -1]
            )

# Add title and axis labels
fig.update_layout(
    yaxis_title="Silhouette Score",
    xaxis_title="# Clusters",
    title="Silhouette Score per cluster"
)

# Render
#fig.show(renderer="notebook")
fig.show() # if using workspace

The optimal K seems to be K=10. Let's check this out

In [27]:
# Instanciate Kmeans with n_clusters=9 and fitting
kmeans = KMeans(n_clusters=10, random_state=0,n_init = 'auto')

# Fit kmeans to our dataset
kmeans.fit(X)

KMeans(n_clusters=10, n_init='auto', random_state=0)

In [28]:
fig = px.scatter_mapbox(sample,lat='Lat',lon='Lon',color=kmeans.labels_+1, # Adding +1 to get clusters from 1 to 9 
                                                                           # instead of 0 to 8
                        zoom=9,mapbox_style="carto-positron",width=800,height=800, 
                        title='KMeans clusters in NYC')
fig.show()

Kmeans clustering is not very efficient. Not any useful information can be highlighted. Let's try with DBSCAN to check if we can get a more detailed analysis.

## **Clustering with DBSCAN**

In [29]:
db = DBSCAN(eps=0.15,metric='manhattan',min_samples=25,algorithm="auto")
db.fit(X)

DBSCAN(eps=0.15, metric='manhattan', min_samples=25)

In [30]:
'''#Ignoring outliers 
sample['db_clusters'] = db.labels_
outliers = (sample['db_clusters']== -1)
sample=sample.loc[~outliers,:]'''

"#Ignoring outliers \nsample['db_clusters'] = db.labels_\noutliers = (sample['db_clusters']== -1)\nsample=sample.loc[~outliers,:]"

In [31]:
fig = px.scatter_mapbox(sample,lat='Lat',lon='Lon',color=db.labels_,
                        zoom=10,mapbox_style="carto-positron",width=800,height=800,
                        labels={'color':'Clusters'},
                        title='DBSCAN clusters in NYC')
fig.show()

**That's quite better ! Some areas stand out clearly :**
1) As expected, Manhattan as a whole. That's not surprinsing. Harlem, in the north part, also seems to be a small cluster.
2) Brooklyn borough gets 2 main clusters : 
    One from Prospect Heights to Brooklyn Dowtown and DUMBO neighbor (very touristic one). 
    The other one is located in the Williamsburg district.  
3) Hunters point Avenue in northwestern Queens appears to be a hotzone. There's the railway station to Long Island here.
4) 3 clusters stand out from the crowd far away from Manhattan : Newark Airport, La Guardia and JFK Airport !!

In [32]:
sample['cluster'] = db.labels_

In [33]:
sample

,Date/Time,Lat,Lon,Base,hour,month,day,dayofweek,cluster
587300,2014-07-31 19:59:00,40.8221,-73.9555,B02617,19,7,31,Thursday,-1
551923,2014-08-26 18:43:00,40.7706,-73.9601,B02617,18,8,26,Tuesday,0
38471,2014-08-01 19:53:00,40.7432,-74.0078,B02598,19,8,1,Friday,0
570574,2014-07-30 16:37:00,40.7592,-73.9792,B02617,16,7,30,Wednesday,0
21778,2014-07-21 14:45:00,40.6419,-73.7883,B02512,14,7,21,Monday,1
...,...,...,...,...,...,...,...,...,...
53018,2014-04-04 17:24:00,40.7700,-73.9672,B02598,17,4,4,Friday,0
1011718,2014-09-28 16:35:00,40.7767,-73.9831,B02764,16,9,28,Sunday,0
313631,2014-07-06 12:36:00,40.7708,-73.9670,B02617,12,7,6,Sunday,0
448037,2014-06-28 22:56:00,40.6877,-73.9781,B02617,22,6,28,Saturday,2


In [34]:

sample_monday = sample.loc[sample['dayofweek']=='Monday',:].sort_values(by='hour')
sample_tuesday = sample.loc[sample['dayofweek']=='Tuesday',:].sort_values(by='hour')
sample_wednesday = sample.loc[sample['dayofweek']=='Wednesday',:].sort_values(by='hour')
sample_thursday = sample.loc[sample['dayofweek']=='Thursday',:].sort_values(by='hour')
sample_friday = sample.loc[sample['dayofweek']=='Friday',:].sort_values(by='hour')
sample_saturday = sample.loc[sample['dayofweek']=='Saturday',:].sort_values(by='hour')
sample_sunday = sample.loc[sample['dayofweek']=='Sunday',:].sort_values(by='hour')

fig1 = px.scatter_mapbox(sample_monday,lat='Lat',lon='Lon',color='cluster',
                        zoom=10,mapbox_style="carto-positron",width=800,height=800,
                        labels={'color':'Clusters'},
                        title='DBSCAN clusters in NYC for mondays')
fig2 = px.scatter_mapbox(sample_tuesday,lat='Lat',lon='Lon',color='cluster',
                        zoom=10,mapbox_style="carto-positron",width=800,height=800,
                        labels={'color':'Clusters'},
                        title='DBSCAN clusters in NYC for tuesdays')
fig3 = px.scatter_mapbox(sample_wednesday,lat='Lat',lon='Lon',color='cluster',
                        zoom=10,mapbox_style="carto-positron",width=800,height=800,
                        labels={'color':'Clusters'},
                        title='DBSCAN clusters in NYC for wednesdays')
fig4 = px.scatter_mapbox(sample_thursday,lat='Lat',lon='Lon',color='cluster',
                        zoom=10,mapbox_style="carto-positron",width=800,height=800,
                        labels={'color':'Clusters'},
                        title='DBSCAN clusters in NYC for thursdays')
fig5 = px.scatter_mapbox(sample_friday,lat='Lat',lon='Lon',color='cluster',
                        zoom=10,mapbox_style="carto-positron",width=800,height=800,
                        labels={'color':'Clusters'},
                        title='DBSCAN clusters in NYC for fridays')
fig6 = px.scatter_mapbox(sample_saturday,lat='Lat',lon='Lon',color='cluster',
                        zoom=10,mapbox_style="carto-positron",width=800,height=800,
                        labels={'color':'Clusters'},
                        title='DBSCAN clusters in NYC for saturdays')
fig7 = px.scatter_mapbox(sample_sunday,lat='Lat',lon='Lon',color='cluster',
                        zoom=10,mapbox_style="carto-positron",width=800,height=800,
                        labels={'color':'Clusters'},
                        title='DBSCAN clusters in NYC for sundays')



In [41]:
display(fig1)

In [42]:
display(fig2)

In [50]:
fig3

In [51]:
fig4

In [53]:
fig5

In [54]:
fig6

In [82]:
sample_density = sample_friday.sort_values(by='hour')
fig = px.density_mapbox(sample_density, lat = 'Lat', lon = 'Lon', z = 'hour', radius = 12,
                        mapbox_style='carto-positron',animation_frame = "hour",
                        width=800, height=600,title = 'Density of rides per hour of day', zoom=9.5
                       )

fig.show()